In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityimports

url = "https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityimports/current/countrybycommodityimports.zip"

In [3]:
r = requests.get(url) 

brexit_FEB21 = zf.ZipFile(io.BytesIO(r.content))

brexit_FEB21.extractall(".\\data")

filename = brexit_FEB21.namelist()

In [4]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Import")

In [5]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,1997JAN,1997FEB,1997MAR,1997APR,1997MAY,1997JUN,1997JUL,...,2020JUL,2020AUG,2020SEP,2020OCT,2020NOV,2020DEC,2021JAN,2021FEB,2021MAR,2021APR
0,0 Food & live animals,AD Andorra,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.07,0.00
1,0 Food & live animals,AE United Arab Emirates,IM Imports,0.27,0.07,2.51,22.37,5.06,0.23,0.61,...,1.48,1.63,0.84,1.02,1.46,1.18,2.19,1.20,5.34,1.54
2,0 Food & live animals,AF Afghanistan,IM Imports,0.06,0.07,0.00,0.23,0.05,0.05,0.00,...,0.00,0.02,0.11,0.04,0.12,0.10,0.11,0.24,0.10,0.12
3,0 Food & live animals,AG Antigua & Barbuda,IM Imports,0.02,0.12,0.00,0.00,0.04,0.01,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02
4,0 Food & live animals,AI Anguilla,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [6]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "imports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.imports = df.imports*1000000

In [7]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"imports": "sum"})

In [8]:
topcntry = test.sort_values(by = "imports", ascending = False).iloc[0:23].index.to_list()

In [9]:
df = df[df.COUNTRY.isin(topcntry)]

In [10]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)
df.COUNTRY.replace({"Extra EU 28 (Rest of World)": "Excl. EU 28 (Rest of World)"}, inplace = True)

In [11]:
grp = df.groupby(["COUNTRY"])

In [12]:
def ex_metal_total(df):
        
    if not (df[df["COMMODITY"] == "9 Unspecified goods"].empty):
                
        foo = pd.DataFrame([])

        fooall = df[df["COMMODITY"] == "Total"].set_index(["time"])["imports"] 
        
        foobar = df[df["COMMODITY"] == "9 Unspecified goods"].set_index(["time"])["imports"]

        foo["imports"] = fooall - foobar
    # Grab the total and then sbutract off the unspecified goods
    
        foo["COUNTRY"] = df.COUNTRY.unique()[0]
    
        #print(df.COUNTRY.unique())
    
        foo["COMMODITY"] = "Total (ex. metals)"
        
        foo.reset_index(inplace = True)
        
        foo[df.columns]
        
        df = df.append(foo)
    
    return df

In [13]:
df = grp.apply(ex_metal_total)

In [14]:
df.drop(["COUNTRY"], axis = 1, inplace = True)

df.reset_index(inplace = True)

df.drop(["level_1"], axis = 1, inplace = True)

In [15]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [16]:
df.head()

imports
COUNTRY COMMODITY                   time                  
Belgium 0 Food & live animals       1997-01-01  44340000.0
        00 Live animals             1997-01-01     70000.0
        01 Meat & meat preparations 1997-01-01   6000000.0
        02 Dairy products & eggs    1997-01-01   5340000.0
        03 Fish & shellfish         1997-01-01    690000.0

In [17]:
out_file = ".\\data"+ "\\UK-imports.parquet"

pq.write_table(pa.Table.from_pandas(df.sort_index()), out_file)

In [18]:
file = ".\\data"+ "\\UK-imports.parquet"

foo = pq.read_table(file).to_pandas()

In [19]:
df.tail()

imports
COUNTRY     COMMODITY          time                    
Whole world Total (ex. metals) 2020-12-01  3.967900e+10
                               2021-01-01  3.095700e+10
                               2021-02-01  3.241000e+10
                               2021-03-01  3.905800e+10
                               2021-04-01  3.616600e+10